## CSV and Excel Parsing  - Structured Data

In [2]:
import pandas as pd
import os

In [3]:
os.makedirs('data/structured_files', exist_ok=True)


In [4]:
# Create sample data frame
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'A high-performance laptop suitable for all your computing needs.',
        'A wireless mouse with ergonomic design.',
        'A mechanical keyboard with customizable backlight.',
        'A 24-inch full HD monitor with vibrant colors.',
        '1080p webcam with noise cancellation.'
    ]
}
df = pd.DataFrame(data)
df.to_csv('data/structured_files/products.csv', index=False)


In [7]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)

    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)